In [7]:
ngrok_auth_token='31lPiYdRTqIlxu66bA5XbBozzWA_3orTomSnRqWHRDy2ufGQK'

In [2]:
!sudo apt -y update
!sudo apt -y install apache2-utils
!sudo apt -y install expect

Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble InRelease
Hit:2 http://archive.ubuntu.com/ubuntu noble InRelease                         
Hit:3 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:4 http://archive.ubuntu.com/ubuntu noble-updates InRelease        
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease    
Hit:6 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:7 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
55 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
apache2-utils is already the newest version (2.4.58-1ubuntu8.8).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state infor

In [3]:
import random
import os
import subprocess
import re

In [ ]:
os.makedirs('/tmp/auth', exist_ok=True)

!htpasswd -b -c /tmp/auth/htpasswd user1 "pass1"
!htpasswd -b /tmp/auth/htpasswd user2 "pass2"
print(f'\nHere is the htpasswd file:\n------------------')
!cat /tmp/auth/htpasswd

/workspaces/MLOps
Adding password for user user1
Adding password for user user2

Here is the htpasswd file:
------------------
user1:$apr1$No5wVtvE$ZFrgsanjGqE7vE23qOGNI0
user2:$apr1$1ExLJf9Y$XjHGY3ZMr3pIy9tLEM/xd0


In [11]:
! docker ps -a
!docker run -d \
  -p 5000:5000 \
  --restart=always \
  --name registry \
  -v /tmp/auth:/auth \
  -e "REGISTRY_AUTH=htpasswd" \
  -e "REGISTRY_AUTH_HTPASSWD_REALM=Registry Realm" \
  -e "REGISTRY_AUTH_HTPASSWD_PATH=/auth/htpasswd" \
  registry:2
! docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


5c507b8e17e0992da5487c8441c3dfbbbc465d55e93bdbcbd424a47fc1e3d6c9
CONTAINER ID   IMAGE        COMMAND                  CREATED                  STATUS                  PORTS                                         NAMES
5c507b8e17e0   registry:2   "/entrypoint.sh /etc…"   Less than a second ago   Up Less than a second   0.0.0.0:5000->5000/tcp, [::]:5000->5000/tcp   registry


In [6]:

!docker exec -it registry sh -c 'set | egrep REGISTRY_'
	# Verify you see the REGISTRY_* environment variables configured above.

REGISTRY_AUTH='htpasswd'
REGISTRY_AUTH_HTPASSWD_PATH='/tmp/auth/htpasswd'
REGISTRY_AUTH_HTPASSWD_REALM='Registry Realm'


In [15]:
print(f'The local file and the one inside the docker registry should be the same.')
print(f'\nHere is the htpasswd file:\n------------------')
!cat /tmp/auth/htpasswd
print(f'\nHere is what is found inside the docker regitry:\n------------------')
!docker exec -it registry sh -c 'cat /auth/htpasswd'
	# Verify the contents of both files (local file and file inside running container) is the same file.

The local file and the one inside the docker registry should be the same.

Here is the htpasswd file:
------------------
user1:$apr1$No5wVtvE$ZFrgsanjGqE7vE23qOGNI0
user2:$apr1$1ExLJf9Y$XjHGY3ZMr3pIy9tLEM/xd0

Here is what is found inside the docker regitry:
------------------
user1:$apr1$No5wVtvE$ZFrgsanjGqE7vE23qOGNI0
user2:$apr1$1ExLJf9Y$XjHGY3ZMr3pIy9tLEM/xd0


In [16]:
print(f'The docker container registry is done. You can now access it via remotely.')

The docker container registry is done. You can now access it via remotely.


In [ ]:
# Locally test the docker container registry.

random_number = random.randint(10000, 99999)
print(f'random_number: {random_number}')
print('')
!curl http://localhost:5000/v2/_catalog
print('')
!docker pull ubuntu
print('')
!docker image ls
print('')
!docker pull ubuntu
print('')
os.system(f'docker tag ubuntu localhost:5000/my-image-{random_number}')
print('')
os.system(f'docker push localhost:5000/my-image-{random_number}')
print('')
print('================')
result = subprocess.run('docker image ls'.split(), capture_output=True, text=True)
print(f'{result.stdout}')
if not re.search(f'localhost:5000/my-image-{random_number}', result.stdout):
    raise('ERROR: It failed')

random_number: 48400

{"errors":[{"code":"UNAUTHORIZED","message":"authentication required","detail":[{"Type":"registry","Class":"","Name":"catalog","Action":"*"}]}]}



Using default tag: latest
latest: Pulling from library/ubuntu
Digest: sha256:7c06e91f61fa88c08cc74f7e1b7c69ae24910d745357e0dfe1d2c0322aaf20f9
Status: Image is up to date for ubuntu:latest
docker.io/library/ubuntu:latest

REPOSITORY                      TAG       IMAGE ID       CREATED         SIZE
localhost:5000/my-image-66314   latest    e0f16e6366fe   4 weeks ago     78.1MB
localhost:5000/my-image-91151   latest    e0f16e6366fe   4 weeks ago     78.1MB
ubuntu                          latest    e0f16e6366fe   4 weeks ago     78.1MB
localhost:5000/my-image-12613   latest    e0f16e6366fe   4 weeks ago     78.1MB
localhost:5000/my-image-13801   latest    e0f16e6366fe   4 weeks ago     78.1MB
localhost:5000/my-image-49860   latest    e0f16e6366fe   4 weeks ago     78.1MB
localhost:5000/my-image-20812   latest    e0f16e6366fe   4 weeks ago     78.1MB
localhost:5000/my-image-87790   latest    e0f16e6366fe   4 weeks ago     78.1MB
localhost:5000/my-image-91190   latest    e0f16e6366fe   4 w

no basic auth credentials


In [4]:
# Use a public URL address
!sudo apt update
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xvzf ngrok-v3-stable-linux-amd64.tgz
!sudo mv ngrok /usr/local/bin/
!sudo chmod +x /usr/local/bin/ngrok
result = subprocess.run('ngrok version'.split(), capture_output=True, text=True)
print(f'{result.stdout}')
if not re.search(f'ngrok version', result.stdout):
    raise('ERROR: It failed')


Hit:1 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble InRelease
Hit:2 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease    
Hit:4 http://archive.ubuntu.com/ubuntu noble InRelease                       0m
Hit:5 http://archive.ubuntu.com/ubuntu noble-updates InRelease               
Hit:6 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
48 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2025-08-27 07:35:43--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 35.71.179.82, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting respons

public_url: "36709704c2d8.ngrok-free.app"
https://36709704c2d8.ngrok-free.app


In [ ]:
!ngrok config add-authtoken $ngrok_auth_token
!nohup ngrok http 5000
!sleep 5




Authtoken saved to configuration file: /home/codespace/.config/ngrok/ngrok.yml


nohup: ignoring input and appending output to 'nohup.out'


In [70]:
import json

cmd39489="curl -s localhost:4040/api/tunnels"
result = subprocess.run(cmd39489.split(), capture_output=True, text=True)
# print(f'output: {result.stdout}')
output = json.loads(result.stdout)
# print(f'(2) output: {output}')
public_url = output['tunnels'][0]['public_url']
public_url = re.sub('.*://', '', public_url)
print(f'\nThis is the ngrok public URL that you will use when accessing docker container registry')
print(f'remotely:')
print(f'\tpublic_url: "{public_url}"')
print(f'\tExamples:')
print(f'\t    % docker login {public_url}')
print(f'\t    % docker push {public_url}/my-tag-4529')


This is the ngrok public URL that you will use when accessing docker container registry
remotely:
	public_url: "36709704c2d8.ngrok-free.app"
	Examples:
	    % docker login 36709704c2d8.ngrok-free.app
	    % docker push 36709704c2d8.ngrok-free.app/my-tag-4529


In [62]:
!sudo groupadd docker
!sudo usermod -aG docker $USER
!nohup newgrp docker
!docker ps -a
    # Verify this works without the 'sudo' command
print('\n----------------------\n')
!docker image ls
    # Verify this works without the 'sudo' command

groupadd: group 'docker' already exists


nohup: ignoring input and appending output to 'nohup.out'
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES

----------------------

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


In [63]:
!docker ps -a
	# Under the IMAGE column, the 'registry' is the instance running the docker container registry.
	# You MUST keep this docker instance running if accessing the docker container registry.

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
